In [6]:
from make_benchmarks import *
import os
sys.path.append("/home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/")
from parser import *

In [7]:
# roots is a list of the root terms
def create_bm_lists_h2h(t0_file, t1_file, t0_ont_graph, t1_ont_graph, t1_minus_ont_graph,roots, c2_t_minus_1_ont_graph, c3_t_minus_1_ont_graph, BM_path = "/data/rashika/CAFA4/head2head/eval/BM_GO/", remove_protein_binding = True):

    #Prop t0 and t1 in their respective ontologies
    t0_prop = process_raw_annot(t0_file, t0_ont_graph, roots)
    t1_prop = process_raw_annot(t1_file, t1_ont_graph, roots)
    
    # Keep terms common in t0_ont and t1_ont
    t0_common, t1_common =  keep_common_go_terms(t0_prop, t1_prop, t0_ont_graph, t1_ont_graph)
    print(len(t0_common), len(t1_common))
    
    # Keep terms common in (t0_ont, t1_ont) and (c2_t_minus_1,, c3_tminus_1) 
    t0_common, t1_common =  keep_common_go_terms(t0_common, t1_common, c2_t_minus_1_ont_graph, c2_t_minus_1_ont_graph)
    print(len(t0_common), len(t1_common))

    # Propagate back in the t(-1) ontology
    subontologies = {aspect: fetch_aspect(t1_minus_ont_graph, roots[aspect]) for aspect in roots}
    t0_eval = propagate_terms(t0_common, subontologies)
    t1_eval = propagate_terms(t1_common, subontologies)

    # Convert the eval Dfs into annotation lists
    t0_annot_list = get_annot_list(t0_eval)
    t1_annot_list = get_annot_list(t1_eval)
    
    # get baselines
    get_baselines(t0_annot_list, t1_annot_list,  BM_path, remove_protein_binding = True)

In [20]:
work_dir = "/data/rashika/CAFA4/"


t0_mapped_path = work_dir + "mapped/2019-12-17/t0_2019.csv"
t1_mapped_path = work_dir + "mapped/2025-03-07/t1_2025.csv"

#t0_mapped_path = work_dir + "mapped/t1_test.csv"
#t1_mapped_path = work_dir + "mapped/t0_test.csv"

t0_ont_file = '/data/rashika/CAFA4/obo/go_2019_12_09/go-basic.obo' 
t0_ont_graph = clean_ontology_edges(obonet.read_obo(t0_ont_file))     

t1_ont_file = "/data/rashika/CAFA4/obo/go_2025-02-06/go-basic.obo"
t1_ont_graph = clean_ontology_edges(obonet.read_obo(t1_ont_file)) 

t_minus_ont_file =  "/data/rashika/CAFA4/obo/go_2019_10_07/go-basic.obo"
t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(t_minus_ont_file))

c3_t_minus_ont_file =  "/data/rashika/CAFA4/head2head/CAFA3/go.obo"
c3_t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(c3_t_minus_ont_file))

c2_t_minus_ont_file =  "/data/rashika/CAFA4/head2head/CAFA2/go.obo"
c2_t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(c2_t_minus_ont_file))


roots = {'BPO': 'GO:0008150', 'CCO': 'GO:0005575', 'MFO': 'GO:0003674'}

GT_2025_H2H_path = work_dir + "/ground_truth_lists/2025-03-07/H2H/all/"

create_bm_lists_h2h(t0_mapped_path,t1_mapped_path, t0_ont_graph, t1_ont_graph, t_minus_1_ont_graph,roots, c2_t_minus_1_ont_graph, c3_t_minus_1_ont_graph, GT_2025_H2H_path)

2947238 2976166
2736840 2741287
/data/rashika/CAFA4//ground_truth_lists/2025-03-07/H2H/all/type1.txt
/data/rashika/CAFA4//ground_truth_lists/2025-03-07/H2H/all/type2.txt
/data/rashika/CAFA4//ground_truth_lists/2025-03-07/H2H/all/type3.txt
/data/rashika/CAFA4//ground_truth_lists/2025-03-07/H2H/all/type12.txt


In [18]:
GT_2025_H2H_path = work_dir + "/ground_truth_lists/2025-03-07/H2H/all/"

### Use the created benchmarks to map to the benchmarks that are in all the CAFAs

In [9]:
h2h_proteins = pd.read_csv('/data/common/CAFA4/h2h/H2H_CAFA4.tsv', sep = '\t', header = 0)
display(h2h_proteins)

,AC ID,Sequence,Taxon ID,CAFA_ID_2,CAFA_ID_3,CAFA_ID_4,Entry_ID_2,Entry_ID_3,Entry_ID_4
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...,10090,T100900000001,T100900000001,T100900000001,1433B_MOUSE,1433B_MOUSE,1433B_MOUSE
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,10090,T100900000002,T100900000002,T100900000002,1433E_MOUSE,1433E_MOUSE,1433E_MOUSE
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,10090,T100900000003,T100900000003,T100900000003,1433F_MOUSE,1433F_MOUSE,1433F_MOUSE
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,10090,T100900000004,T100900000004,T100900000004,1433G_MOUSE,1433G_MOUSE,1433G_MOUSE
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...,10090,T100900000005,T100900000005,T100900000005,1433S_MOUSE,1433S_MOUSE,1433S_MOUSE
...,...,...,...,...,...,...,...,...,...
76514,Q8ZNV7,MTSLVSLENVSVSFGQRRVLSDVSLELSPGKILTLLGPNGAGKSTL...,99287,T992870001767,T992870001785,T992870001793,ZNUC_SALTY,ZNUC_SALTY,ZNUC_SALTY
76515,Q9L9I0,MKRNNKSAIALIALSLLALSSGAAFAGHHWGNNDGMWQQGGSPLTT...,99287,T992870001768,T992870001786,T992870001794,ZRAP_SALTY,ZRAP_SALTY,ZRAP_SALTY
76516,P25852,MIRGKIDILVVDDDVSHCTILQALLRGWGYNVALAYSGHDALAQVR...,99287,T992870001769,T992870001787,T992870001795,ZRAR_SALTY,ZRAR_SALTY,ZRAR_SALTY
76517,P37461,MSFIRLHKDAAATWLSRLLPAAIFILVGLFSIMVIRDYGRESAAAR...,99287,T992870001770,T992870001788,T992870001796,ZRAS_SALTY,ZRAS_SALTY,ZRAS_SALTY


In [21]:
gt_df

,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T2848120001425,GO:0120114,CCO,T2848120001415,T2848120001391,T2848120001425
1,T2848120001425,GO:0005688,CCO,T2848120001415,T2848120001391,T2848120001425
2,T2848120001425,GO:0005682,CCO,T2848120001415,T2848120001391,T2848120001425
3,T2848120001425,GO:0097525,CCO,T2848120001415,T2848120001391,T2848120001425
4,T2848120001425,GO:0005686,CCO,T2848120001415,T2848120001391,T2848120001425
...,...,...,...,...,...,...
441171,T100900006985,GO:0010557,BPO,T100900006962,T100900006930,T100900006985
441172,T100900013725,GO:0016755,MFO,T100900013568,T100900013428,T100900013725
441173,T100900013725,GO:0016746,MFO,T100900013568,T100900013428,T100900013725
441174,T101160003635,GO:0009987,BPO,T101160003590,T101160003571,T101160003635


In [19]:
GT_list = os.listdir(GT_2025_H2H_path)
mapping_4_to_3 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_3']))
mapping_4_to_2 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_2']))
mapping_4_to_4 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_4']))
for gt_file in GT_list:
    print(gt_file)
    gt_df = pd.read_csv(GT_2025_H2H_path+gt_file, header = None, sep = '\t')
    gt_df.columns = ['ID', 'GO ID', 'aspect']
    
    # Shortlist the proteins to the h2h common proteins
    gt_df = gt_df[gt_df['ID'].isin(h2h_proteins['CAFA_ID_4'])].copy()
    
    # Map to IDs from respective CAFAs
    gt_df['CAFA3_ID'] = gt_df['ID'].map(mapping_4_to_3)
    gt_df['CAFA2_ID'] = gt_df['ID'].map(mapping_4_to_2)
    gt_df['CAFA4_ID'] = gt_df['ID'].map(mapping_4_to_4)
    
    # Get the benchmark copy for respective CAFA
    gt_df_cafa4 = gt_df.loc[:, ["CAFA4_ID", "GO ID", "aspect"]].copy()
    gt_df_cafa3 = gt_df.loc[:, ["CAFA3_ID", "GO ID", "aspect"]].copy()
    gt_df_cafa2 = gt_df.loc[:, ["CAFA2_ID", "GO ID", "aspect"]].copy()
    
    # Make the directory to save the mapped benchmarks for each cafa
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa4')
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa3')
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa2')
    gt_df_cafa4.to_csv('/data/rashika/CAFA4/ground_truth_lists/2025-03-07/H2H/cafa4/' + gt_file, sep = '\t', header = None, index = False)
    gt_df_cafa3.to_csv('/data/rashika/CAFA4/ground_truth_lists/2025-03-07/H2H/cafa3/' + gt_file, sep = '\t', header = None, index = False)
    gt_df_cafa2.to_csv('/data/rashika/CAFA4/ground_truth_lists/2025-03-07/H2H/cafa2/' + gt_file, sep = '\t', header = None, index = False)
    
    display(gt_df)

type12.txt


,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T96060016506,GO:0003674,MFO,T96060016419,T96060016427,T96060016506
1,T96060016506,GO:0005488,MFO,T96060016419,T96060016427,T96060016506
2,T96060016506,GO:0005515,MFO,T96060016419,T96060016427,T96060016506
3,T37020010489,GO:0003674,MFO,T37020009691,T37020007890,T37020010489
4,T37020010489,GO:0005488,MFO,T37020009691,T37020007890,T37020010489
...,...,...,...,...,...,...
160573,T100900005740,GO:1901576,BPO,T100900005730,T100900005694,T100900005740
160574,T100900005740,GO:0006753,BPO,T100900005730,T100900005694,T100900005740
160575,T100900005740,GO:0071704,BPO,T100900005730,T100900005694,T100900005740
160576,T100900005740,GO:0034641,BPO,T100900005730,T100900005694,T100900005740


type1.txt


,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T96060016506,GO:0003674,MFO,T96060016419,T96060016427,T96060016506
1,T96060016506,GO:0005488,MFO,T96060016419,T96060016427,T96060016506
2,T96060016506,GO:0005515,MFO,T96060016419,T96060016427,T96060016506
3,T37020010489,GO:0003674,MFO,T37020009691,T37020007890,T37020010489
4,T37020010489,GO:0005488,MFO,T37020009691,T37020007890,T37020010489
...,...,...,...,...,...,...
77592,T100900001170,GO:0005622,CCO,T100900001155,T100900001111,T100900001170
77593,T100900001170,GO:0000407,CCO,T100900001155,T100900001111,T100900001170
77594,T100900001170,GO:0005575,CCO,T100900001155,T100900001111,T100900001170
77595,T100900001170,GO:0110165,CCO,T100900001155,T100900001111,T100900001170


type2.txt


,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T96060016588,GO:0003674,MFO,T96060016500,T96060016507,T96060016588
1,T96060016588,GO:0005488,MFO,T96060016500,T96060016507,T96060016588
2,T96060016588,GO:0005515,MFO,T96060016500,T96060016507,T96060016588
3,T96060019779,GO:0003674,MFO,T96060019546,T96060019602,T96060019779
4,T96060019779,GO:0005488,MFO,T96060019546,T96060019602,T96060019779
...,...,...,...,...,...,...
82976,T100900005740,GO:1901576,BPO,T100900005730,T100900005694,T100900005740
82977,T100900005740,GO:0006753,BPO,T100900005730,T100900005694,T100900005740
82978,T100900005740,GO:0071704,BPO,T100900005730,T100900005694,T100900005740
82979,T100900005740,GO:0034641,BPO,T100900005730,T100900005694,T100900005740


type3.txt


,ID,GO ID,aspect,CAFA3_ID,CAFA2_ID,CAFA4_ID
0,T2848120001425,GO:0120114,CCO,T2848120001415,T2848120001391,T2848120001425
1,T2848120001425,GO:0005688,CCO,T2848120001415,T2848120001391,T2848120001425
2,T2848120001425,GO:0005682,CCO,T2848120001415,T2848120001391,T2848120001425
3,T2848120001425,GO:0097525,CCO,T2848120001415,T2848120001391,T2848120001425
4,T2848120001425,GO:0005686,CCO,T2848120001415,T2848120001391,T2848120001425
...,...,...,...,...,...,...
441171,T100900006985,GO:0010557,BPO,T100900006962,T100900006930,T100900006985
441172,T100900013725,GO:0016755,MFO,T100900013568,T100900013428,T100900013725
441173,T100900013725,GO:0016746,MFO,T100900013568,T100900013428,T100900013725
441174,T101160003635,GO:0009987,BPO,T101160003590,T101160003571,T101160003635


### Map Known DF to IDs from each CAFA to generate respective known_dfs

In [31]:
known_df = pd.read_csv(t0_mapped_path, header = None, sep = '\t')
known_df.columns = ['ID', 'GO ID', 'aspect']

# Shortlist the proteins to the h2h common proteins
known_df = known_df[known_df['ID'].isin(h2h_proteins['CAFA_ID_4'])].copy()
    
# Map to IDs from respective CAFAs
known_df['CAFA3_ID'] = known_df['ID'].map(mapping_4_to_3)
known_df['CAFA2_ID'] = known_df['ID'].map(mapping_4_to_2)
known_df['CAFA4_ID'] = known_df['ID'].map(mapping_4_to_4)
    
# Get the benchmark copy for respective CAFA
known_df_cafa4 = known_df.loc[:, ["CAFA4_ID", "GO ID", "aspect"]].copy()
known_df_cafa3 = known_df.loc[:, ["CAFA3_ID", "GO ID", "aspect"]].copy()
known_df_cafa2 = known_df.loc[:, ["CAFA2_ID", "GO ID", "aspect"]].copy()

known_df_cafa4.to_csv('/data/rashika/CAFA4/ground_truth_lists/2025-03-07/H2H/cafa4/known.csv', sep = '\t', header = None, index = False)
known_df_cafa3.to_csv('/data/rashika/CAFA4/ground_truth_lists/2025-03-07/H2H/cafa3/known.csv', sep = '\t', header = None, index = False)
known_df_cafa2.to_csv('/data/rashika/CAFA4/ground_truth_lists/2025-03-07/H2H/cafa2/known.csv', sep = '\t', header = None, index = False)


    